In [3]:
import pandas as pd
import pyodbc 
from joblib import dump, load

import os
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

email = 'revosoft6@gmail.com'
senha = 'vxxqsdeshpdherzy'

In [26]:
#pip install pandas
#pip install pyodbc
#pip install joblib
#pip install scikit-learn


In [4]:
dados_conexao = (
    "Driver={SQL Server};"
    "Server=DESKTOP-FETGEDT;"
    "Database=REVOSOFT;"
)

conexao = pyodbc.connect(dados_conexao)

cursor = conexao.cursor()

print('Conectado com o banco de dados')

Conectado
Conectado com o banco de dados


In [45]:
query = """select * from Sensores where VeiculosId = 1"""
df = pd.read_sql(query, conexao)
print("buscando informações do banco")

C:\Users\guilh\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [5]:
def encontrarUltimo():
    
    select = """  SELECT TOP (1) * FROM [Revosoft].[dbo].[Pecas] WHERE VeiculosId = 1 ORDER BY PecasId DESC"""
    cursor.execute(select)
    ultimo = cursor.fetchone()
    return ultimo


(20, Decimal('100.00'), Decimal('100.00'), Decimal('100.00'), 1, Decimal('100.00'))


In [49]:
def buscaUsuario(idVeiculo):
    
    select = f"""  SELECT TOP (1) *, [VeiculosId] FROM [dbo].[Veiculos] WHERE [VeiculosId] ={idVeiculo}"""
    cursor.execute(select)
    veiculo = cursor.fetchone()
    print (veiculo[4])
    
    select2 = f"""SELECT TOP (1) * from [Revosoft].[dbo].[Usuarios] WHERE [UsuariosId] = {veiculo[4]}"""
    cursor.execute(select2)
    usuario = cursor.fetchone()
    
    return usuario, veiculo

1


((1, 'Felipe', 'Gabriel', '123456789', 'feehlipeeh014@hotmail.com', '11947214217'),
 (1, 'RVS-2022', 'Ranger', '2022/2022', 1, 1))

In [33]:
def score_temperatura(ultimo_score):
    for i in range(len(df)):
        if df['Temperatura'].loc[i] >= 101 or df['Temperatura'].loc[i] <= 83:    
            return(float(ultimo_score)-(float(ultimo_score)*(0.025)))
    return(ultimo_score)

In [34]:
def score_kms(ultimo_score, kms):
    kms_score=ultimo_score
    for i in range(len(df)):
        if df['Kms'].loc[i] >= kms:
            kms= df['Kms'].loc[i] + 100
            kms_score= (float(ultimo_score)-(float(ultimo_score)*0.02))
    return(kms_score)

In [35]:
def score_tempo(ultimo_score, tempo):
    for i in range(len(df)):
        if df['Tempo'].loc[i] >= tempo:
            tempo= df['Tempo'].loc[i] + 182
            return(float(ultimo_score)-(float(ultimo_score)*0.25))
    return(ultimo_score)

In [36]:
def score_bateria(ultimo_score, tempo):
    for i in range(len(df)):
        if df['Tempo'].loc[i] >= tempo:
            tempo= df['Tempo'].loc[i] + 2
            return(float(ultimo_score)-(float(ultimo_score)*0.02))
    return(ultimo_score)

In [50]:
print(Tratando os dados...)

97.5 98.0 100.00 98.0


In [51]:
def inserirValores(temperatura, kms, tempo, bateria, veiculo_id):
    inserir = f"""INSERT INTO Pecas (MotorScore, CambioScore, PneuScore, VeiculosId, BateriaScore) VALUES 
          ({tempo}, {temperatura}, {tempo}, {veiculo_id}, {bateria} )"""      
    
    cursor.execute(inserir)
    cursor.commit()
    
    print("informações inseridas no banco com sucesso")
inserirValores(score_temperatura(encontrarUltimo()[2]), score_kms(encontrarUltimo()[3], 100), score_tempo(encontrarUltimo()[1], 182), score_bateria(encontrarUltimo()[5], 2), 1)    

INSERT INTO Pecas (MotorScore, CambioScore, PneuScore, VeiculosId, BateriaScore) VALUES 
          (100.00, 97.5, 100.00, 1, 98.0 )
inserido com sucesso


In [39]:
modelo = load('C:/Users/guilh/OneDrive/Documents/Revosoft/IA/FORD_NEXT_IA/IA/Modelo_IA.joblib')

modelo.predict(([[0.8,0.5,0.9,0.7]]))

C:\Users\guilh\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.2 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\guilh\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.0.2 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\guilh\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestCl

array(['Revisao'], dtype=object)

Note: you may need to restart the kernel to use updated packages.


ERROR: unknown command "unpickle"



In [40]:
def IA_Email(score):
    resposta = modelo.predict(([[score[1], score[2], score[3], score[4]]]))
    
    if (resposta == "Bom"):
        
        usuario = buscaUsuario(score[4])
        
        msg = MIMEMultipart()
        msg['Subject'] = 'REVOSOFT ANALISE'
        msg['From'] = email
        msg['To'] = usuario[0][4]
        mensagem = emailTemplate(resposta[0],usuario[1][1],usuario[1][2], usuario[1][3], usuario[0][1])
        
        msg.attach(MIMEText(mensagem, 'html'))  
           
        with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
            smtp.login(email,senha)
            smtp.send_message(msg)
            smtp.quit()
            print("Avalição enviada para o email do Usuario")
       
    return resposta

print(IA_Email(encontrarUltimo()))

1


C:\Users\guilh\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


['Bom']
